In [1]:
import pandas as pd
import sklearn.feature_extraction.text as tx, sklearn.naive_bayes as nb, sklearn.linear_model as lm, sklearn.ensemble as en, sklearn.metrics as me, sklearn.model_selection as ms
from warnings import filterwarnings as fw
fw('ignore')

все слова с учетом регистра<br>
все в одном регистре<br>
без стоп слов<br>
выделение специфичных для классов<br>
-- -- 
лог регрессия, случайный лес, байес

In [2]:
ord('Н')%3

0

In [3]:
def fitpredict(xtrain,ytrain,xtest,ytest):
    
    lr =  lm.LogisticRegression()
    rfc = en.RandomForestClassifier()
    cnb = nb.ComplementNB()

    lr.fit(xtrain,ytrain)
    rfc.fit(xtrain,ytrain)
    cnb.fit(xtrain,ytrain)

    pred_lr = lr.predict(xtest)
    pred_rfc = rfc.predict(xtest)
    pred_cnb = cnb.predict(xtest)
    
    print('Логистическая регрессия: \n', me.classification_report(ytest, pred_lr, digits=5),
        'Случайный лес: \n', me.classification_report(ytest, pred_rfc, digits=5),
        'Наивный Байес: \n', me.classification_report(ytest, pred_cnb, digits=5))
    
    return lr, rfc, cnb

In [4]:
corpus = pd.read_csv('SMSpam.csv')
corpus['Message'] = corpus['Message'].str.replace(r'[^\w\s]+','')

In [5]:
cnt, cntS, cntH = 0, 0, 0
for i in corpus['Message']:
    cnt += len(i.split(' '))
for i in corpus[corpus['Category']=='spam']['Message']:
    cntS += len(i.split(' '))
for i in corpus[corpus['Category']=='ham']['Message']:
    cntH += len(i.split(' '))
print('Всего строк:', corpus.shape[0], '\nИз них', 
    corpus['Category'].value_counts()['spam'], 'spam и',corpus['Category'].value_counts()['ham'], 'ham', 
    '\nВсего слов: ', cnt, '\nиз них', cntS, 'spam и', cntH, 'ham')

Всего строк: 5572 
Из них 747 spam и 4825 ham 
Всего слов:  87265 
из них 17788 spam и 69477 ham


In [6]:
xtrain, xtest, ytrain, ytest = ms.train_test_split(corpus['Message'],corpus['Category'], test_size=0.2)
print('Обучающая выборка:', xtrain.shape,'\nТестовая выборка', xtest.shape)

Обучающая выборка: (4457,) 
Тестовая выборка (1115,)


-- -- 
Используем все слова с учетом регистра:

In [7]:
cv = tx.CountVectorizer(lowercase=False)
cv.fit(corpus['Message'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 11596 токенов


In [8]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97862   0.99896   0.98868       962
        spam    0.99248   0.86275   0.92308       153

    accuracy                        0.98027      1115
   macro avg    0.98555   0.93085   0.95588      1115
weighted avg    0.98052   0.98027   0.97968      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97172   1.00000   0.98566       962
        spam    1.00000   0.81699   0.89928       153

    accuracy                        0.97489      1115
   macro avg    0.98586   0.90850   0.94247      1115
weighted avg    0.97560   0.97489   0.97380      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99457   0.95218   0.97292       962
        spam    0.76289   0.96732   0.85303       153

    accuracy                        0.95426      1115
   macro avg    0.87873   0.95975   0.91297      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем все слова, но в одном регистре:

In [9]:
cv = tx.CountVectorizer()
cv.fit(corpus['Message'])
print('Всего', len(cv.vocabulary_.keys()), 'токенов')

tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 9542 токенов


In [10]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97864   1.00000   0.98920       962
        spam    1.00000   0.86275   0.92632       153

    accuracy                        0.98117      1115
   macro avg    0.98932   0.93137   0.95776      1115
weighted avg    0.98157   0.98117   0.98057      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96976   1.00000   0.98465       962
        spam    1.00000   0.80392   0.89130       153

    accuracy                        0.97309      1115
   macro avg    0.98488   0.90196   0.93798      1115
weighted avg    0.97391   0.97309   0.97184      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99041   0.96570   0.97789       962
        spam    0.81356   0.94118   0.87273       153

    accuracy                        0.96233      1115
   macro avg    0.90198   0.95344   0.92531      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Найдем все стоп-слова, которые встречаются в большом количестве сообщений (более 500 , что около 9% всей выборки) и уберем их:

In [33]:
cv = tx.CountVectorizer(max_df = 500)
cv.fit(corpus['Message'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов')
print(stop)

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 14 токенов
{'to', 'me', 'is', 'of', 'for', 'have', 'and', 'it', 'the', 'your', 'you', 'call', 'my', 'in'}


In [34]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97566   1.00000   0.98768       962
        spam    1.00000   0.84314   0.91489       153

    accuracy                        0.97848      1115
   macro avg    0.98783   0.92157   0.95129      1115
weighted avg    0.97900   0.97848   0.97769      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97074   1.00000   0.98515       962
        spam    1.00000   0.81046   0.89531       153

    accuracy                        0.97399      1115
   macro avg    0.98537   0.90523   0.94023      1115
weighted avg    0.97475   0.97399   0.97282      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99245   0.95634   0.97406       962
        spam    0.77660   0.95425   0.85630       153

    accuracy                        0.95605      1115
   macro avg    0.88452   0.95529   0.91518      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

Найдем все стоп-слова, которые встречаются в маленьком количестве сообщений (менее 10, что около 0.18% всей выборки) и уберем:

In [43]:
cv = tx.CountVectorizer(min_df = 10)
cv.fit(corpus['Message'])
stop = cv.stop_words_
print('Всего', len(stop), 'токенов')
#print(stop) // не будем выводить список слов, так как он слишком большой

cv = tx.CountVectorizer(analyzer = 'word', stop_words = stop)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

Всего 8562 токенов


In [42]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.98063   1.00000   0.99022       962
        spam    1.00000   0.87582   0.93380       153

    accuracy                        0.98296      1115
   macro avg    0.99032   0.93791   0.96201      1115
weighted avg    0.98329   0.98296   0.98248      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97467   1.00000   0.98717       962
        spam    1.00000   0.83660   0.91103       153

    accuracy                        0.97758      1115
   macro avg    0.98734   0.91830   0.94910      1115
weighted avg    0.97815   0.97758   0.97672      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98934   0.96466   0.97684       962
        spam    0.80791   0.93464   0.86667       153

    accuracy                        0.96054      1115
   macro avg    0.89862   0.94965   0.92175      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

Используем встроенный список стоп-слов:

In [13]:
cv = tx.CountVectorizer(stop_words = 'english')
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.97764   1.00000   0.98869       962
        spam    1.00000   0.85621   0.92254       153

    accuracy                        0.98027      1115
   macro avg    0.98882   0.92810   0.95561      1115
weighted avg    0.98071   0.98027   0.97962      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.96878   1.00000   0.98414       962
        spam    1.00000   0.79739   0.88727       153

    accuracy                        0.97220      1115
   macro avg    0.98439   0.89869   0.93571      1115
weighted avg    0.97307   0.97220   0.97085      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.99549   0.91788   0.95511       962
        spam    0.65351   0.97386   0.78215       153

    accuracy                        0.92556      1115
   macro avg    0.82450   0.94587   0.86863      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем только слова с высокой важностью:

In [14]:
cv = tx.CountVectorizer()
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

e = en.RandomForestClassifier()
e.fit(tr_xtrain,ytrain)
e.score(tr_xtest,ytest)

0.9748878923766816

In [15]:
imp = pd.DataFrame({'name' : cv.vocabulary_.keys(), 'importance' : e.feature_importances_}) 
imp = imp.sort_values(by = 'importance', ascending = False)
top100 = imp.head(n=1000)['name'].to_numpy()

cv = tx.CountVectorizer(vocabulary = top100)
cv.fit(corpus['Message'])
tr_xtrain = cv.transform(xtrain)
tr_xtest = cv.transform(xtest)

fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.93281   0.99584   0.96330       962
        spam    0.95455   0.54902   0.69710       153

    accuracy                        0.93453      1115
   macro avg    0.94368   0.77243   0.83020      1115
weighted avg    0.93580   0.93453   0.92677      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.95972   0.99064   0.97494       962
        spam    0.92623   0.73856   0.82182       153

    accuracy                        0.95605      1115
   macro avg    0.94297   0.86460   0.89838      1115
weighted avg    0.95512   0.95605   0.95393      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.97098   0.90437   0.93649       962
        spam    0.57991   0.83007   0.68280       153

    accuracy                        0.89417      1115
   macro avg    0.77545   0.86722   0.80964      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())

-- -- 
Используем tf-idf векторизатор:

In [16]:
tfidfv = tx.TfidfVectorizer()
tfidfv.fit(corpus['Message'])
tr_xtrain = tfidfv.transform(xtrain)
tr_xtest = tfidfv.transform(xtest)

In [17]:
fitpredict(tr_xtrain, ytrain, tr_xtest, ytest)

Логистическая регрессия: 
               precision    recall  f1-score   support

         ham    0.96004   0.99896   0.97911       962
        spam    0.99123   0.73856   0.84644       153

    accuracy                        0.96323      1115
   macro avg    0.97563   0.86876   0.91278      1115
weighted avg    0.96432   0.96323   0.96091      1115
 Случайный лес: 
               precision    recall  f1-score   support

         ham    0.97172   1.00000   0.98566       962
        spam    1.00000   0.81699   0.89928       153

    accuracy                        0.97489      1115
   macro avg    0.98586   0.90850   0.94247      1115
weighted avg    0.97560   0.97489   0.97380      1115
 Наивный Байес: 
               precision    recall  f1-score   support

         ham    0.98519   0.96778   0.97640       962
        spam    0.81765   0.90850   0.86068       153

    accuracy                        0.95964      1115
   macro avg    0.90142   0.93814   0.91854      1115
weighted avg 

(LogisticRegression(), RandomForestClassifier(), ComplementNB())